In [1]:
import pandas as pd
import pyproj
import xarray as xr
from loaddea import loaddea
from burnmapping import severitymapping,geomedian,nbrdist,cosdist
import time
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import xarray as xr
%matplotlib inline

In [3]:
testsite=18
sites = pd.read_csv('sites.txt')
wgs84 = pyproj.Proj(init='epsg:4326')
gda94 = pyproj.Proj(init='epsg:3577')

sites = pd.read_csv('sites.txt')
easting,northing = pyproj.transform(wgs84,gda94,sites.centre_longitude[testsite],sites.centre_latitude[testsite])

print("Load data for {} \n".format(sites.region_name[testsite]))
lon = (sites.centre_longitude[testsite]-0.125,sites.centre_longitude[testsite]+0.125) # min to max
lat = (sites.centre_latitude[testsite]-0.125,sites.centre_latitude[testsite]+0.125)
x = (easting-12500,easting+12500)
y = (northing+12500,northing-12500)

if sites.year[testsite]>=2013:
    datatime = ('2013-01-01',str(sites.year[testsite])+'-12-31')
    reftime = ('2013-01-01',str(sites.year[testsite]-1)+'-12-31')# period used for the calculation of geometric median
    sensor = 8
else:
    datatime = (str(sites.year[testsite]-5)+'-01-01',str(sites.year[testsite])+'-12-31')
    reftime = (str(sites.year[testsite]-5)+'-01-01',str(sites.year[testsite]-1)+'-12-31')
    sensor = 5
res = (-25,25) # resolution in meters
print(lon,lat,reftime)
severity_period =(str(sites.year[testsite])+'-01-01',str(sites.year[testsite])+'-12-31') # period of interest for change/severity mapping
method = 3 # change detection method



Load data for Sumac 

(144.72983400000001, 144.97983400000001) (-41.269222999999997, -41.019222999999997) ('2013-01-01', '2015-12-31')


In [4]:
sites.region_name[testsite]

'Sumac'

In [ ]:
start_time = time.monotonic()
data= loaddea(x=x,y=y,reftime=datatime,resolution=res,landsat_numbers=[sensor]) #load cloud free data for the given region
#data = data.astype('float16')
print("---{} minutes for loading data.---".format((time.monotonic()-start_time)/60))

Geomedian and cosdist

In [ ]:
start_time = time.monotonic()
out=geomedian(data.sel(time=slice(reftime[0],reftime[1])), ['red','green','blue','nir','swir1','swir2'], epsilon=10, max_iter=40)# calculate the geometrice medians and the distances
print("---{} minutes for geomedian and cosdist calculation.---".format((time.monotonic()-start_time)/60))


In [ ]:
#remove the data for the reference period
data = data.sel(time=slice(severity_period[0],severity_period[1]))

In [ ]:
start_time = time.monotonic()
cos_dist = cosdist(data, out.geomedian.astype('float32'))
print("---{} minutes for cos dist.---".format((time.monotonic()-start_time)/60))


In [ ]:
start_time = time.monotonic()
nbr_dist=nbrdist(data, out.geomedian )
print("---{} minutes for nbr dist.---".format((time.monotonic()-start_time)/60))

In [ ]:
Dist = xr.Dataset(coords={'time':data.time,'y':data.y[:],'x':data.x[:]}, attrs={'crs':'EPSG:3577'})
Dist['cosdist'] = (('time','y','x'),cos_dist.cosdist.data)
Dist['NBR'] = (('time','y','x'),nbr_dist.NBR.data)
Dist['NBRDist'] = (('time','y','x'),nbr_dist.NBRdist.data)
Dist['ChangeDir'] = (('time','y','x'),nbr_dist.ChangeDir.data)
Dist['NBRoutlier'] = (('y','x'),out.NBRoutlier)
Dist['CDistoutlier'] = (('y','x'),out.CDistoutlier)


In [ ]:
del nbr_dist, cos_dist,out

In [ ]:
start_time = time.monotonic()
output=severitymapping(Dist,period=(severity_period[0],severity_period[1]),method=2,growing=True)
print("---{} minutes for severity.---".format((time.monotonic()-start_time)/60))


In [ ]:
output.Severe.plot.imshow(cmap='binary')